Carpeta https://www.football-data.co.uk/spainm.php

Combinar los csv

In [3]:
import pandas as pd
import os

# Ruta de la carpeta donde están los archivos CSV
folder_path = "odds"

# Lista para almacenar los DataFrames
dfs = []

# Leer todos los archivos en la carpeta
for file in sorted(os.listdir(folder_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        try:
            # Intentar leer el CSV manejando errores de formato
            df = pd.read_csv(file_path, on_bad_lines='skip', encoding='utf-8', engine='python')
            df["Season"] = file.replace(".csv", "")  # Agregar columna de temporada
            dfs.append(df)
        except Exception as e:
            print(f"Error leyendo {file}: {e}")

# Unir todos los DataFrames asegurando que todas las columnas estén incluidas
combined_df = pd.concat(dfs, ignore_index=True, sort=False)

# Guardar el resultado en un nuevo archivo CSV
combined_df.to_csv("combined_odds.csv", index=False)

print("Archivo combinado guardado como 'combined_odds.csv'")


Archivo combinado guardado como 'combined_odds.csv'


Verificar que todos los equipos son de primera división

Columan "Div" valor SP1

Borrar esa columna

In [4]:
# Cargar el archivo combinado
file_path = "combined_odds.csv"
df = pd.read_csv(file_path)

# Verificar los valores únicos en la columna 'Div'
if "Div" in df.columns and df["Div"].nunique() == 1 and df["Div"].iloc[0] == "SP1":
    df.drop(columns=["Div"], inplace=True)

df.head()

C:\Users\anaig\AppData\Local\Temp\ipykernel_7376\3581885593.py:3: DtypeWarning: Columns (100) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,GBH,...,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA
0,09/09/00,Barcelona,Malaga,2,1,H,2,0,H,1.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,09/09/00,La Coruna,Ath Bilbao,2,0,H,0,0,D,1.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09/09/00,Real Madrid,Valencia,2,1,H,0,0,D,1.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,09/09/00,Sociedad,Santander,2,2,D,0,0,D,1.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,09/09/00,Zaragoza,Espanol,1,2,A,0,0,D,1.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Información del dataset

Información básica del partido:
- Div → Liga (SP1 = LaLiga)
- Date → Fecha del partido
- Time → Hora del partido
- HomeTeam → Equipo local
- AwayTeam → Equipo visitante


Resultados del partido:
- FTHG → Goles del equipo local (Full Time Home Goals)
- FTAG → Goles del equipo visitante (Full Time Away Goals)
- FTR → Resultado del partido (H = Victoria local, D = Empate, A = Victoria visitante)
- HTHG → Goles del equipo local al descanso (Half Time Home Goals)
- HTAG → Goles del equipo visitante al descanso (Half Time Away Goals)
- HTR → Resultado al descanso (H, D, A)


Estadísticas avanzadas del partido: (No siempre disponibles)
- HS, AS → Disparos del equipo local y visitante
- HST, AST → Disparos a puerta del equipo local y visitante
- HC, AC → Córners del equipo local y visitante
- HF, AF → Faltas cometidas por el equipo local y visitante
- HY, AY → Tarjetas amarillas del equipo local y visitante
- HR, AR → Tarjetas rojas del equipo local y visitante

Cuotas de apuestas: (Probabilidades para diferentes mercados)
- 1X2 (Ganador del partido)
    - B365H, B365D, B365A → Cuotas de Bet365 para local, empate y visitante
    - PSH, PSD, PSA → Cuotas de Pinnacle
    - WHH, WHD, WHA → Cuotas de William Hill
    - AvgH, AvgD, AvgA → Cuotas promedio del mercado

- Over/Under (Total de goles)
    - B365>2.5, B365<2.5 → Cuotas para más/menos de 2.5 goles en Bet365
    - Max>2.5, Max<2.5 → Máximas cuotas del mercado para más/menos de 2.5 goles
    - Avg>2.5, Avg<2.5 → Cuotas promedio del mercado


- Hándicap asiático
    - B365AHH, B365AHA → Cuotas de Bet365 para hándicap asiático
    - BbMxAHH, BbMxAHA → Máximas cuotas del mercado
    - AvgAHH, AvgAHA → Cuotas promedio del mercado

### Columnas que vamos a utilizar
Datos del partido
- Date, Season HomeTeam, AwayTeam
- FTHG, FTAG, FTR (como variable objetivo si predices goles en lugar de resultado)

Cuotas de apuestas (indicadores de probabilidad según el mercado)
- 1X2 → Promedio de cuotas (AvgH, AvgD, AvgA)
- Hándicap asiático → Puede indicar qué equipo es favorito según las apuestas


In [6]:
import pandas as pd
import numpy as np


# Eliminar columnas "Unnamed" si existen
df = df.loc[:, ~df.columns.str.contains('Unnamed')]

# Convertir la columna Date a tipo datetime
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors='coerce')

# Crear la columna Season según la fecha del partido
def get_season(date):
    year = date.year
    if date.month >= 8:  # Si es agosto o después, pertenece a la temporada actual
        return f"{year}-{str(year + 1)[-2:]}"
    else:  # Si es antes de agosto, pertenece a la temporada anterior
        return f"{year - 1}-{str(year)[-2:]}"


df["Season"] = df["Date"].apply(get_season)

# **Lista de cuotas disponibles en las casas de apuestas**
bookmaker_odds = {
    "AvgH": ["B365H", "PSH", "WHH", "IWH", "LBH", "VCH", "SJH", "SYH", "GBH"],
    "AvgD": ["B365D", "PSD", "WHD", "IWD", "LBD", "VCD", "SJD", "SYD", "GBD"],
    "AvgA": ["B365A", "PSA", "WHA", "IWA", "LBA", "VCA", "SJA", "SYA", "GBA"],
    "AvgAHH": ["B365AHH", "GBAHH", "LBAHH"],
    "AvgAHA": ["B365AHA", "GBAHA", "LBAHA"]   
}

# **Lista de cuotas de comparadores (BetBrain) como segunda opción**
betbrain_odds = {
    "AvgAHH": ["BbAvAHH", "BbMxAHH"],
    "AvgAHA": ["BbAvAHA", "BbMxAHA"]
}

# **Calcular la media de las cuotas usando solo casas de apuestas reales**
for avg_col, alt_cols in bookmaker_odds.items():
    available_cols = [col for col in alt_cols if col in df.columns]  # Filtrar solo columnas que existen
    if available_cols:
        df[avg_col] = df[available_cols].mean(axis=1).round(2)  # Calcular la media y redondear a 2 decimales

# **Si no hay datos de casas reales, usar los valores de BetBrain**
for avg_col, alt_cols in betbrain_odds.items():
    if avg_col in df.columns:  # Asegurar que la columna objetivo existe
        for col in alt_cols:
            if col in df.columns:
                df[avg_col] = df[avg_col].fillna(df[col].round(2))  # Rellenar con BetBrain si está disponible

# **Seleccionar las columnas que vamos a utilizar**
columns_to_keep = [
    "Date", "Season", "HomeTeam", "AwayTeam",
    "FTHG", "FTAG", "FTR",  # Resultados del partido
    "AvgH", "AvgD", "AvgA",  # Cuotas 1X2
    "AvgAHH", "AvgAHA"  # Cuotas Hándicap Asiático
]

# **Filtrar solo las columnas necesarias**
df = df[columns_to_keep]

# **Mostrar resumen de valores NaN por columna**
nan_summary = df.isna().sum()
print("Resumen de valores NaN por columna después del procesamiento:")
print(nan_summary[nan_summary > 0])

# **Mostrar las primeras filas para verificar**
print(df.head())


Resumen de valores NaN por columna después del procesamiento:
AvgH         2
AvgD         2
AvgA         2
AvgAHH    1159
AvgAHA    1267
dtype: int64
        Date   Season     HomeTeam    AwayTeam  FTHG  FTAG FTR  AvgH  AvgD  \
0 2000-09-09  2000-01    Barcelona      Malaga     2     1   H  1.29  4.11   
1 2000-09-09  2000-01    La Coruna  Ath Bilbao     2     0   H  1.54  3.31   
2 2000-09-09  2000-01  Real Madrid    Valencia     2     1   H  1.61  3.30   
3 2000-09-09  2000-01     Sociedad   Santander     2     2   D  1.79  3.20   
4 2000-09-09  2000-01     Zaragoza     Espanol     1     2   A  1.70  3.15   

   AvgA  AvgAHH  AvgAHA  
0  7.98     NaN     NaN  
1  5.30     NaN     NaN  
2  4.45     NaN     NaN  
3  3.68     NaN     NaN  
4  4.18     NaN     NaN  


In [7]:
# Contar valores NaN por temporada en las columnas afectadas
nan_per_season = df.groupby("Season")[["AvgH", "AvgD", "AvgA", "AvgAHH", "AvgAHA"]].apply(lambda x: x.isna().sum())

# Mostrar el resumen
print(nan_per_season)

         AvgH  AvgD  AvgA  AvgAHH  AvgAHA
Season                                   
2000-01     1     1     1     380     380
2001-02     0     0     0     380     380
2002-03     1     1     1     380     380
2003-04     0     0     0       8      14
2004-05     0     0     0       0       0
2005-06     0     0     0       1       6
2006-07     0     0     0       0       2
2007-08     0     0     0       0     100
2008-09     0     0     0       0       1
2009-10     0     0     0       0       1
2010-11     0     0     0       0       3
2011-12     0     0     0       0       0
2012-13     0     0     0       0       0
2013-14     0     0     0       0       0
2014-15     0     0     0       0       0
2015-16     0     0     0       0       0
2016-17     0     0     0       0       0
2017-18     0     0     0       0       0
2018-19     0     0     0       0       0
2019-20     0     0     0       8       0
2020-21     0     0     0       0       0
2021-22     0     0     0       1 

In [8]:
# Filtrar desde la temporada 2003-04 en adelante y eliminar la 2024-25
df = df[(df["Season"] >= "2003-04") & (df["Season"] != "2024-25")]
df 

,Date,Season,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AvgH,AvgD,AvgA,AvgAHH,AvgAHA
1140,2003-08-30,2003-04,Albacete,Osasuna,0,2,A,2.21,3.06,2.99,2.05,1.80
1141,2003-08-30,2003-04,Ath Bilbao,Barcelona,0,1,A,2.64,3.13,2.42,1.77,2.08
1142,2003-08-30,2003-04,Espanol,Sociedad,1,1,D,2.58,3.10,2.48,1.87,1.98
1143,2003-08-30,2003-04,Malaga,Villarreal,0,0,D,2.27,3.08,2.88,2.07,1.78
1144,2003-08-30,2003-04,Real Madrid,Betis,2,1,H,1.38,4.00,7.18,1.94,1.91
...,...,...,...,...,...,...,...,...,...,...,...,...
9115,2024-05-25,2023-24,Real Madrid,Betis,0,0,D,5.90,2.30,6.03,2.00,1.89
9116,2024-05-26,2023-24,Getafe,Mallorca,1,2,A,3.01,2.51,3.25,2.01,1.88
9117,2024-05-26,2023-24,Celta,Valencia,2,2,D,3.36,2.27,3.59,1.97,1.92
9118,2024-05-26,2023-24,Las Palmas,Alaves,1,1,D,2.75,2.63,3.28,1.81,2.09


In [9]:
# Contar valores NaN por temporada en las columnas afectadas
nan_per_season = df.groupby("Season")[["AvgH", "AvgD", "AvgA", "AvgAHH", "AvgAHA"]].apply(lambda x: x.isna().sum())

# Mostrar el resumen
print(nan_per_season)

         AvgH  AvgD  AvgA  AvgAHH  AvgAHA
Season                                   
2003-04     0     0     0       8      14
2004-05     0     0     0       0       0
2005-06     0     0     0       1       6
2006-07     0     0     0       0       2
2007-08     0     0     0       0     100
2008-09     0     0     0       0       1
2009-10     0     0     0       0       1
2010-11     0     0     0       0       3
2011-12     0     0     0       0       0
2012-13     0     0     0       0       0
2013-14     0     0     0       0       0
2014-15     0     0     0       0       0
2015-16     0     0     0       0       0
2016-17     0     0     0       0       0
2017-18     0     0     0       0       0
2018-19     0     0     0       0       0
2019-20     0     0     0       8       0
2020-21     0     0     0       0       0
2021-22     0     0     0       1       0
2022-23     0     0     0       0       0
2023-24     0     0     0       0       0


In [1]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv("../datasets/combined_odds_cleaned.csv")

# Eliminar las columnas especificadas
df.drop(columns=['AvgAHH', 'AvgAHA'], inplace=True)

# Guardar el DataFrame limpio
df.to_csv("../datasets/combined_odds_cleaned.csv", index=False)


In [10]:
# Guardar en un archivo CSV
df.to_csv("combined_odds_cleaned.csv", index=False)

print("Archivo guardado como combined_odds_cleaned.csv")

Archivo guardado como combined_odds_cleaned.csv


In [3]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv("../datasets/combined_odds_cleaned.csv")
df

,Date,Season,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AvgH,AvgD,AvgA
0,2003-08-30,2003-04,Albacete,Osasuna,0,2,A,2.21,3.06,2.99
1,2003-08-30,2003-04,Ath Bilbao,Barcelona,0,1,A,2.64,3.13,2.42
2,2003-08-30,2003-04,Espanol,Sociedad,1,1,D,2.58,3.10,2.48
3,2003-08-30,2003-04,Malaga,Villarreal,0,0,D,2.27,3.08,2.88
4,2003-08-30,2003-04,Real Madrid,Betis,2,1,H,1.38,4.00,7.18
...,...,...,...,...,...,...,...,...,...,...
7975,2024-05-25,2023-24,Real Madrid,Betis,0,0,D,5.90,2.30,6.03
7976,2024-05-26,2023-24,Getafe,Mallorca,1,2,A,3.01,2.51,3.25
7977,2024-05-26,2023-24,Celta,Valencia,2,2,D,3.36,2.27,3.59
7978,2024-05-26,2023-24,Las Palmas,Alaves,1,1,D,2.75,2.63,3.28


In [4]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv("../datasets/dataset_transformado.csv")
df

,season,date,team,rival_team,home_adv,last_season_team,last_season_rival,pct_wins,avg_goals_scored,avg_goals_received,...,pct_wins_vs_rival,avg_goals_scored_vs_rival,avg_goals_received_vs_rival,goal_difference_vs_rival,AvgH,AvgD,AvgA,goals_team,goals_rival,result
0,2004-05,2004-08-28,Ath Madrid,Malaga,1,7,10,0.4,1.7,1.9,...,0.5,1.5,1.5,0,1.75,3.20,4.30,2,0,1
1,2004-05,2004-08-28,Espanol,La Coruna,1,17,3,0.5,1.7,1.4,...,0.5,1.5,1.0,1,2.80,3.16,2.32,1,1,0
2,2004-05,2004-08-28,Numancia,Betis,1,21,9,0.0,0.0,0.0,...,0.0,0.0,0.0,0,2.50,3.12,2.56,1,1,0
3,2004-05,2004-08-29,Mallorca,Real Madrid,1,11,4,0.6,1.9,1.7,...,0.5,2.0,2.5,-1,4.42,3.25,1.75,0,1,-1
4,2004-05,2004-08-29,Osasuna,Ath Bilbao,1,12,5,0.2,1.0,1.3,...,0.0,1.0,1.5,-1,2.31,3.13,2.77,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,2023-24,2024-05-25,Almeria,Cadiz,1,17,16,0.2,1.2,1.9,...,0.0,1.0,1.0,0,3.09,2.39,3.61,6,1,1
7596,2023-24,2024-05-26,Las Palmas,Alaves,1,21,21,0.0,0.3,1.7,...,0.2,0.6,1.6,-5,2.75,2.63,3.28,1,1,0
7597,2023-24,2024-05-26,Getafe,Mallorca,1,15,9,0.2,0.5,1.1,...,0.4,0.8,0.6,1,3.01,2.51,3.25,1,2,-1
7598,2023-24,2024-05-26,Celta,Valencia,1,13,14,0.5,1.4,1.6,...,0.0,0.4,1.8,-7,3.36,2.27,3.59,2,2,0


In [6]:

# Primer bloque: 7 columnas
print("\n--- Columnas 1 a 7 ---\n")
display(df.iloc[:, 0:7].head())

# Segundo bloque: columnas 8 a 12 (5 columnas)
print("\n--- Columnas 8 a 12 ---\n")
display(df.iloc[:, 7:12].head())

# Tercer bloque: columnas 13 a 17 (5 columnas)
print("\n--- Columnas 13 a 17 ---\n")
display(df.iloc[:, 12:17].head())

# Cuarto bloque: el resto
print(f"\n--- Columnas 18 a {df.shape[1]} ---\n")
display(df.iloc[:, 17:].head())


--- Columnas 1 a 7 ---



,season,date,team,rival_team,home_adv,last_season_team,last_season_rival
0,2004-05,2004-08-28,Ath Madrid,Malaga,1,7,10
1,2004-05,2004-08-28,Espanol,La Coruna,1,17,3
2,2004-05,2004-08-28,Numancia,Betis,1,21,9
3,2004-05,2004-08-29,Mallorca,Real Madrid,1,11,4
4,2004-05,2004-08-29,Osasuna,Ath Bilbao,1,12,5



--- Columnas 8 a 12 ---



,pct_wins,avg_goals_scored,avg_goals_received,goal_difference,pct_wins_rival
0,0.4,1.7,1.9,-2,0.3
1,0.5,1.7,1.4,3,0.6
2,0.0,0.0,0.0,0,0.4
3,0.6,1.9,1.7,2,0.3
4,0.2,1.0,1.3,-3,0.4



--- Columnas 13 a 17 ---



,avg_goals_scored_rival,avg_goals_received_rival,goal_difference_rival,pct_wins_vs_rival,avg_goals_scored_vs_rival
0,1.2,1.3,-1,0.5,1.5
1,1.5,0.7,8,0.5,1.5
2,1.1,1.3,-2,0.0,0.0
3,1.6,2.3,-7,0.5,2.0
4,1.8,1.4,4,0.0,1.0



--- Columnas 18 a 25 ---



,avg_goals_received_vs_rival,goal_difference_vs_rival,AvgH,AvgD,AvgA,goals_team,goals_rival,result
0,1.5,0,1.75,3.20,4.30,2,0,1
1,1.0,1,2.80,3.16,2.32,1,1,0
2,0.0,0,2.50,3.12,2.56,1,1,0
3,2.5,-1,4.42,3.25,1.75,0,1,-1
4,1.5,-1,2.31,3.13,2.77,1,1,0


In [9]:
# Imprimir bloques específicos de columnas
# Primer bloque: primeras 7 columnas
print("\n--- Columnas 1 a 7 ---\n")
display(df.iloc[:, 0:7].head())


--- Columnas 1 a 7 ---



,season,date,team,rival_team,home_adv,last_season_team,last_season_rival
0,2004-05,2004-08-28,Ath Madrid,Malaga,1,7,10
1,2004-05,2004-08-28,Espanol,La Coruna,1,17,3
2,2004-05,2004-08-28,Numancia,Betis,1,21,9
3,2004-05,2004-08-29,Mallorca,Real Madrid,1,11,4
4,2004-05,2004-08-29,Osasuna,Ath Bilbao,1,12,5


In [10]:
# Segundo bloque: siguientes 6 columnas
print("\n--- Columnas 8 a 13 ---\n")
display(df.iloc[:, 7:13].head())

# Tercer bloque: siguientes 4 columnas
print("\n--- Columnas 14 a 17 ---\n")
display(df.iloc[:, 13:17].head())

# Último bloque: el resto de columnas
print("\n--- Columnas 18 al final ---\n")
display(df.iloc[:, 17:].head())


--- Columnas 8 a 13 ---



,pct_wins,avg_goals_scored,avg_goals_received,goal_difference,pct_wins_rival,avg_goals_scored_rival
0,0.4,1.7,1.9,-2,0.3,1.2
1,0.5,1.7,1.4,3,0.6,1.5
2,0.0,0.0,0.0,0,0.4,1.1
3,0.6,1.9,1.7,2,0.3,1.6
4,0.2,1.0,1.3,-3,0.4,1.8



--- Columnas 14 a 17 ---



,avg_goals_received_rival,goal_difference_rival,pct_wins_vs_rival,avg_goals_scored_vs_rival
0,1.3,-1,0.5,1.5
1,0.7,8,0.5,1.5
2,1.3,-2,0.0,0.0
3,2.3,-7,0.5,2.0
4,1.4,4,0.0,1.0



--- Columnas 18 al final ---



,avg_goals_received_vs_rival,goal_difference_vs_rival,AvgH,AvgD,AvgA,goals_team,goals_rival,result
0,1.5,0,1.75,3.20,4.30,2,0,1
1,1.0,1,2.80,3.16,2.32,1,1,0
2,0.0,0,2.50,3.12,2.56,1,1,0
3,2.5,-1,4.42,3.25,1.75,0,1,-1
4,1.5,-1,2.31,3.13,2.77,1,1,0
